In [1]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


Welcome to JupyROOT 6.24/04


In [3]:
path_to_fit_result_files = '/work/halld/home/viducic/data/fit_params/{}/tf1_gaus_cross_section_values.csv'
df_pipkmks = pd.read_csv(path_to_fit_result_files.format('pipkmks'))
df_pimkpks = pd.read_csv(path_to_fit_result_files.format('pimkpks'))

pipkmks_grouped = df_pipkmks.groupby('beam_energy')
pimkpks_grouped = df_pimkpks.groupby('beam_energy')

In [ ]:
pipkmks_graphs = []
for beam_energy, group in pipkmks_grouped:
    graph = ROOT.TGraphErrors(len(group.t_bin_middle.to_numpy()), group.t_bin_middle.to_numpy(dtype='d'), group.cross_section.to_numpy(), group.t_bin_width.to_numpy(), group.cross_section_error.to_numpy())
